In [ ]:
import pandas as pd
import numpy as np
final_ledger = pd.read_csv('../data/final_ledger.csv')
max_drawdown = 1-final_ledger['portfolio_value'].min()/100000
print(f'Maximum drawdown: {max_drawdown:.2%}')
## calculate the sharpe rati
blotter= pd.read_csv('../data/blotter_converted.csv')
strategy_counts = blotter['strategy'].value_counts()
final_ledger['daily_return'] = final_ledger['portfolio_value'].pct_change()
final_ledger['daily_return'].iloc[0] = 0  # 设置第一个值为0
sharpe_ratio = final_ledger['daily_return'].mean() / final_ledger['daily_return'].std()*np.sqrt((len(blotter)-1))


# 除以 2，因为每个日期有两行（LEVI 和 MHK）
mean_reversion_count = strategy_counts.get('mean_reversion', 0) // 2
trend_following_count = strategy_counts.get('trend_following', 0) // 2

# 打印结果
print(f"mean_reversion: {mean_reversion_count}")
print(f"trend_following: {trend_following_count}")
print(f"sharpe_ratio: {sharpe_ratio:.2f}")

Maximum drawdown: 4.79%


C:\Users\Xingjian Yin\AppData\Local\Temp\ipykernel_37664\3403295353.py:10: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  final_ledger['daily_return'].iloc[0] = 0  # 设置第一个值为0
C:\Users\Xingjian Yin\AppData\Local\Temp\ipykernel_37664\3403295353

KeyError: 'daily_return'

In [23]:
import pandas as pd

ledger = pd.read_csv('../data/ledger_with_positions.csv')
ledger['position']= ledger['cumulative_position']
ledger.drop(columns=['cumulative_position'], inplace=True)
print(ledger.head())
ledger.to_csv('../data/ledger_with_position.csv', index=False)

         date ticker action   price  quantity  commission        cash  \
0  2024-04-01   LEVI    BUY   20.55         1    0.010275   79.439725   
1  2024-04-01    MHK   SELL  123.29         1    0.061645  202.668080   
2  2024-04-02   LEVI   SELL   20.50         1    0.010250  223.157830   
3  2024-04-02    MHK    BUY  125.15         1    0.062575   97.945255   
4  2024-04-03   LEVI    BUY   20.09         1    0.010045   77.845210   

   portfolio_value  position  
0        95.819725         1  
1       104.038080        -1  
2       108.147830         0  
3        97.945255         0  
4        94.225210         1  


In [24]:
import pandas as pd

new_ledger = pd.read_csv('../data/new_ledger.csv')
# new_ledger_columns = [date, LEVI, MHK, LEVI_POSITION, MHK_POSITION, cumulative_pnl, portfolio_value] ]
cols = ['date', 'ticker', 'action', 'price', 'quantity', 'commission', 'cash', 'portfolio_value', 'position']
ledger = pd.DataFrame(columns=cols)
for i in range(len(new_ledger)):
    ledger.at[2*i, 'date'] = new_ledger['date'][i]
    ledger.at[2*i+1, 'date'] = new_ledger['date'][i]
# ledger: ticker=LEVI if even, otherwise ticker=MHK
for i in range(len(ledger)):
    if i % 2 == 0:
        ticker = 'LEVI'
    else:
        ticker = 'MHK'
    ledger.at[i, 'ticker'] = ticker
# ledger: if even: action= BUY if LEVI_position> LEVI_position_prev, otherwise action=SELL
# ledger: if odd: action= BUY if MHK_position> MHK_position_prev, otherwise action=SELL
for i in range(len(ledger)):
    if i==0:
        ledger.at[i, 'action'] = 'NONE'
        ledger.at[i, 'quantity'] = 0
        ledger.at[i, 'price'] = new_ledger['LEVI'][0]
        ledger.at[i, 'commission'] = 0
        ledger.at[i, 'cash'] = 100000
        ledger.at[i, 'portfolio_value'] = 100000
        ledger.at[i, 'position'] = 0
        continue
    if i==1:
        ledger.at[i, 'action'] = 'NONE'
        ledger.at[i, 'quantity'] = 0
        ledger.at[i, 'price'] = new_ledger['MHK'][0]
        ledger.at[i, 'commission'] = 0
        ledger.at[i, 'cash'] = 100000
        ledger.at[i, 'portfolio_value'] = 100000
        ledger.at[i, 'position'] = 0
        continue
    if i % 2 == 0:
        if new_ledger['LEVI_position'][i//2] > new_ledger['LEVI_position'][i//2-1]:
            ledger.at[i, 'action'] = 'BUY'
            ledger.at[i, 'quantity'] = new_ledger['LEVI_position'][i//2] - new_ledger['LEVI_position'][i//2-1]
            ledger.at[i, 'price'] = new_ledger['LEVI'][i//2]
            ledger.at[i, 'commission'] = 0.005 * new_ledger['LEVI'][i//2] * ledger.at[i, 'quantity']
            ledger.at[i, 'cash'] = ledger.at[i-1, 'cash'] - ledger.at[i, 'price'] * ledger.at[i, 'quantity'] - ledger.at[i, 'commission']
            ledger.at[i, 'portfolio_value'] = new_ledger['portfolio_value'][i//2]
            ledger.at[i, 'position'] = new_ledger['LEVI_position'][i//2]
        elif new_ledger['LEVI_position'][i//2] < new_ledger['LEVI_position'][i//2-1]:
            ledger.at[i, 'action'] = 'SELL'
            ledger.at[i, 'quantity'] = new_ledger['LEVI_position'][i//2-1] - new_ledger['LEVI_position'][i//2]
            ledger.at[i, 'price'] = new_ledger['LEVI'][i//2]
            ledger.at[i, 'commission'] = 0.005 * new_ledger['LEVI'][i//2] * ledger.at[i, 'quantity']
            ledger.at[i, 'cash'] = ledger.at[i-1, 'cash'] + ledger.at[i, 'price'] * ledger.at[i, 'quantity'] - ledger.at[i, 'commission']
            ledger.at[i, 'portfolio_value'] = new_ledger['portfolio_value'][i//2]
            ledger.at[i, 'position'] = new_ledger['LEVI_position'][i//2]
        else:

            ledger.at[i, 'action'] = 'NONE'
            ledger.at[i, 'quantity'] = 0
            ledger.at[i, 'price'] = new_ledger['LEVI'][i//2]
            ledger.at[i, 'commission'] = 0
            ledger.at[i, 'cash'] = ledger.at[i-1, 'cash']
            ledger.at[i, 'portfolio_value'] = new_ledger['portfolio_value'][i//2]
            ledger.at[i, 'position'] = new_ledger['LEVI_position'][i//2]
 
    else:
        if new_ledger['MHK_position'][i//2] > new_ledger['MHK_position'][i//2-1]:
            ledger.at[i, 'action'] = 'BUY'
            ledger.at[i, 'quantity'] = new_ledger['MHK_position'][i//2] - new_ledger['MHK_position'][i//2-1]
            ledger.at[i, 'price'] = new_ledger['MHK'][i//2]
            ledger.at[i, 'commission'] = 0.005 * new_ledger['MHK'][i//2] * ledger.at[i, 'quantity']
            ledger.at[i, 'cash'] = ledger.at[i-1, 'cash'] - ledger.at[i, 'price'] * ledger.at[i, 'quantity'] - ledger.at[i, 'commission']
            ledger.at[i, 'portfolio_value'] = new_ledger['portfolio_value'][i//2]
            ledger.at[i, 'position'] = new_ledger['MHK_position'][i//2]
        elif new_ledger['MHK_position'][i//2] < new_ledger['MHK_position'][i//2-1]:
            ledger.at[i, 'action'] = 'SELL'
            ledger.at[i, 'quantity'] = new_ledger['MHK_position'][i//2-1] - new_ledger['MHK_position'][i//2]
            ledger.at[i, 'price'] = new_ledger['MHK'][i//2]
            ledger.at[i, 'commission'] = 0.005 * new_ledger['MHK'][i//2] * ledger.at[i, 'quantity']
            ledger.at[i, 'cash'] = ledger.at[i-1, 'cash'] + ledger.at[i, 'price'] * ledger.at[i, 'quantity'] - ledger.at[i, 'commission']
            ledger.at[i, 'portfolio_value'] = new_ledger['portfolio_value'][i//2]
            ledger.at[i, 'position'] = new_ledger['MHK_position'][i//2]
            
        else:
            ledger.at[i, 'action'] = 'NONE'
            ledger.at[i, 'quantity'] = 0
            ledger.at[i, 'price'] = new_ledger['MHK'][i//2]
            ledger.at[i, 'commission'] = 0
            ledger.at[i, 'cash'] = ledger.at[i-1, 'cash']
            ledger.at[i, 'portfolio_value'] = new_ledger['portfolio_value'][i//2]
            ledger.at[i, 'position'] = new_ledger['MHK_position'][i//2]
            



print(ledger)
ledger.to_csv('../data/final_ledger.csv', index=False)

           date ticker action   price quantity commission        cash  \
0    2024-04-01   LEVI   NONE   20.55        0          0      100000   
1    2024-04-01    MHK   NONE  123.29        0          0      100000   
2    2024-04-02   LEVI   NONE    20.5        0          0      100000   
3    2024-04-02    MHK   NONE  125.15        0          0      100000   
4    2024-04-03   LEVI   NONE   20.09        0          0      100000   
..          ...    ...    ...     ...      ...        ...         ...   
489  2025-03-24    MHK   NONE  113.74        0          0  94705.4503   
490  2025-03-25   LEVI   NONE   15.59        0          0  94705.4503   
491  2025-03-25    MHK   NONE  114.18        0          0  94705.4503   
492  2025-03-26   LEVI   NONE   16.38        0          0  94705.4503   
493  2025-03-26    MHK   NONE  115.01        0          0  94705.4503   

    portfolio_value position  
0            100000        0  
1            100000        0  
2          100000.0        0  

In [27]:
## renew blotter_converted.csv based on final_ledger.csv
## columns = date,ticker,action,price,quantity,commission,strategy,zscore
collumns = ['date', 'ticker', 'action', 'price', 'quantity', 'commission', 'strategy', 'zscore']
blotter = pd.DataFrame(columns=collumns)
final_ledger = pd.read_csv('../data/final_ledger.csv')
blotter['date'] = final_ledger['date']
blotter['ticker'] = final_ledger['ticker']
blotter['action'] = final_ledger['action']
blotter['price'] = final_ledger['price']
blotter['quantity'] = final_ledger['quantity']
blotter['commission'] = final_ledger['commission']
b =pd.read_csv('../data/b.csv')
for i in range(len(blotter)):
    blotter.at[i, 'strategy'] = b.at[i//2, 'strategy']
    blotter.at[i, 'zscore'] = b.at[i//2, 'zscore']
print(blotter)
blotter.to_csv('../data/blotter_converted.csv', index=False)

           date ticker action   price  quantity  commission strategy    zscore
0    2024-04-01   LEVI   NONE   20.55         0         0.0     none  0.943031
1    2024-04-01    MHK   NONE  123.29         0         0.0     none  0.943031
2    2024-04-02   LEVI   NONE   20.50         0         0.0     none  0.764991
3    2024-04-02    MHK   NONE  125.15         0         0.0     none  0.764991
4    2024-04-03   LEVI   NONE   20.09         0         0.0     none  1.010862
..          ...    ...    ...     ...       ...         ...      ...       ...
489  2025-03-24    MHK   NONE  113.74         0         0.0     none   -0.5437
490  2025-03-25   LEVI   NONE   15.59         0         0.0     none -0.613945
491  2025-03-25    MHK   NONE  114.18         0         0.0     none -0.613945
492  2025-03-26   LEVI   NONE   16.38         0         0.0     none  -0.31571
493  2025-03-26    MHK   NONE  115.01         0         0.0     none  -0.31571

[494 rows x 8 columns]


In [2]:
import pandas as pd

# 读取 blotter_converted.csv 和 b.csv
blotter = pd.read_csv('../data/blotter_converted.csv')
b = pd.read_csv('../data/b.csv')

# 确保 b.csv 的索引是 0 到 n
b.reset_index(inplace=True)  # 将索引重置为 0 到 n
b.rename(columns={'index': 'day_index'}, inplace=True)  # 重命名索引列为 day_index

# 将 blotter_converted.csv 中的日期映射到 b.csv 的索引
# 假设 blotter_converted.csv 中的日期列为 'date'
blotter['day_index'] = blotter['date'].rank(method='dense').astype(int) - 1  # 将日期映射为 0 到 n 的索引

# 使用 b.csv 中的 zscore 替换 blotter_converted.csv 中的 zscore
blotter['zscore'] = blotter['day_index'].map(b.set_index('day_index')['zscore'])

blotter.drop(columns=['day_index'], inplace=True)  # 删除 day_index 列

# 检查修复后的数据
print(blotter.head())


# 保存修复后的 blotter_converted.csv
blotter.to_csv('../data/blotter_converted_fixed.csv', index=False)

         date ticker action   price  quantity  commission        strategy  \
0  2024-04-01   LEVI    BUY   20.55         1    0.010275  mean_reversion   
1  2024-04-01    MHK   SELL  123.29         1    0.061645  mean_reversion   
2  2024-04-02   LEVI   SELL   20.50         1    0.010250  mean_reversion   
3  2024-04-02    MHK    BUY  125.15         1    0.062575  mean_reversion   
4  2024-04-03   LEVI    BUY   20.09         1    0.010045  mean_reversion   

     zscore  
0  0.943031  
1  0.943031  
2  0.764991  
3  0.764991  
4  1.010862  


         cash  portfolio_value
0   79.439725        95.819725
1  202.668080       104.038080
2  223.157830       108.147830
3   97.945255        97.945255
4   77.845210        94.225210


In [6]:
import pandas as pd

# 读取 blotter.csv 文件
blotter = pd.read_csv('../data/blotter.csv')

# 统计每种 strategy 的数量
strategy_counts = blotter['strategy'].value_counts()

# 除以 2，因为每个日期有两行（LEVI 和 MHK）
mean_reversion_count = strategy_counts.get('mean_reversion', 0) // 2
trend_following_count = strategy_counts.get('trend_following', 0) // 2

# 打印结果
print(f"mean_reversion: {mean_reversion_count}")
print(f"trend_following: {trend_following_count}")

mean_reversion: 177
trend_following: 36


In [2]:
import pandas as pd

# 假设您有一个交易日历，包含所有交易日
# 这里以 2020-04-01 到 2025-04-01 的交易日为例
# 您可以从数据源（如交易所或 API）获取实际的交易日历
all_trading_days = pd.date_range(start="2024-04-01", end="2025-04-01", freq="B")  # 'B' 表示工作日

# 读取 blotter.csv 文件
blotter = pd.read_csv('../data/blotter.csv')

# 将交易日编号映射到实际日期
blotter['date'] = blotter['date'].apply(lambda x: all_trading_days[x-989])

# 查看转换后的数据
print(blotter.head())

# 保存回 CSV 文件（如果需要）
blotter.to_csv('../data/blotter_converted.csv', index=False)

        date ticker action   price  quantity  commission        strategy  \
0 2024-04-01   LEVI    BUY   20.55         1    0.010275  mean_reversion   
1 2024-04-01    MHK   SELL  123.29         1    0.061645  mean_reversion   
2 2024-04-02   LEVI   SELL   20.50         1    0.010250  mean_reversion   
3 2024-04-02    MHK    BUY  125.15         1    0.062575  mean_reversion   
4 2024-04-03   LEVI    BUY   20.09         1    0.010045  mean_reversion   

        zscore  
0 -1109.910866  
1 -1109.910866  
2 -1130.937649  
3 -1130.937649  
4 -1075.123099  


In [4]:
ledger = pd.read_csv('../data/ledger.csv')
ledger['date'] = ledger['date'].apply(lambda x: all_trading_days[x-989])
print(ledger.head())
ledger.to_csv('../data/ledger_converted.csv', index=False)

        date ticker action   price  quantity  commission           cash  \
0 2024-04-01   LEVI    BUY   20.55         1    0.010275   99979.439725   
1 2024-04-01    MHK   SELL  123.29         1    0.061645  100102.668080   
2 2024-04-02   LEVI   SELL   20.50         1    0.010250  100123.157830   
3 2024-04-02    MHK    BUY  125.15         1    0.062575   99997.945255   
4 2024-04-03   LEVI    BUY   20.09         1    0.010045   99977.845210   

   portfolio_value  
0     99995.819725  
1    100004.038080  
2    100008.147830  
3     99997.945255  
4     99994.225210  


In [6]:
import pandas as pd

# 读取 ledger.csv 文件
ledger = pd.read_csv('../data/ledger_converted.csv')

# 假设 ledger 文件包含以下列：['date', 'ticker', 'action', 'quantity']
# 将奇数行和偶数行分开处理
ledger['position'] = 0

# 遍历每一行，根据 'action' 更新 'position'
for i in range(len(ledger)):
    if ledger.loc[i, 'action'] == 'BUY':
        ledger.loc[i, 'position'] += ledger.loc[i, 'quantity']
    elif ledger.loc[i, 'action'] == 'SELL':
        ledger.loc[i, 'position'] -= ledger.loc[i, 'quantity']

# 分别计算奇数行和偶数行的累积头寸
ledger['cumulative_position'] = ledger.groupby(ledger.index % 2)['position'].cumsum()

# 查看处理后的数据
print(ledger.head())

# 保存回 CSV 文件（如果需要）
ledger.to_csv('../data/ledger_with_positions.csv', index=False)

         date ticker action   price  quantity  commission           cash  \
0  2024-04-01   LEVI    BUY   20.55         1    0.010275   99979.439725   
1  2024-04-01    MHK   SELL  123.29         1    0.061645  100102.668080   
2  2024-04-02   LEVI   SELL   20.50         1    0.010250  100123.157830   
3  2024-04-02    MHK    BUY  125.15         1    0.062575   99997.945255   
4  2024-04-03   LEVI    BUY   20.09         1    0.010045   99977.845210   

   portfolio_value  position  cumulative_position  
0     99995.819725         1                    1  
1    100004.038080        -1                   -1  
2    100008.147830        -1                    0  
3     99997.945255         1                    0  
4     99994.225210         1                    1  


In [ ]:
'''ArithmeticError




now provide me with a good look html templates for showing our pairs trading methods, I need an interactive graph ( move mouse to show specific number) of the everyday ledger and boltter



'''